In [1]:
import os 
import groq

from dotenv import find_dotenv, load_dotenv
from langchain_groq import ChatGroq
load_dotenv(find_dotenv())
groq.api_key = os.getenv("GROQ_API_KEY")

def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

tools = [add, multiply, divide]
llm = ChatGroq(model="qwen/qwen3-32b")

# For this,  we set parallel tool calling to false as math generally is done sequentially
# the OpenAI model specifically defaults to parallel tool calling for efficiency
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

In [2]:
# create our LLM and prompt it with the overall desired agent behavior
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

# Node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

In [3]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
# Connect the Tools node back to the Assistant, forming a loop.

# After the assistant node executes, tools_condition checks if the model's output is a tool call.
# If it is a tool call, the flow is directed to the tools node.
# The tools node connects back to assistant.
# This loop continues as long as the model decides to call tools.
# If the model response is not a tool call, the flow is directed to END, terminating the process
builder.add_edge("tools", "assistant")
react_graph = builder.compile()

# Show
#display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [4]:
messages = [HumanMessage(content="Add 3 and 4. Multiply the output by 2. Divide the output by 5")]
messages = react_graph.invoke({"messages": messages})

In [5]:
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4. Multiply the output by 2. Divide the output by 5
================================== Ai Message ==================================
Tool Calls:
  add (sma5ez269)
 Call ID: sma5ez269
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================
Tool Calls:
  multiply (4vdfg9w4q)
 Call ID: 4vdfg9w4q
  Args:
    a: 7
    b: 2
================================= Tool Message =================================
Name: multiply

14
================================== Ai Message ==================================
Tool Calls:
  divide (601h7a4de)
 Call ID: 601h7a4de
  Args:
    a: 14
    b: 5
================================= Tool Message =================================
Name: divide

2.8
================================== Ai Message ==================================

T